# plmol Usage Examples

This notebook demonstrates the complete functionality of the plmol library:

1. **PDB Standardization** - Clean and standardize protein structures
2. **Protein Featurization** - Extract atom/residue-level features
3. **Hierarchical Featurization** - ESM embeddings with atom-residue mapping
4. **Molecule Featurization** - Descriptors, fingerprints, and graph features
5. **Protein-Ligand Interaction** - Detect and featurize interactions

We use the PDB structure **10gs** (Glutathione S-transferase) as an example.

In [ ]:
import sys
import os

# Add parent directory to path
sys.path.insert(0, os.path.dirname(os.getcwd()))

from plmol import (
    PDBStandardizer,
    ProteinFeaturizer,
    LigandFeaturizer,
    PLInteractionFeaturizer,
    HierarchicalFeaturizer,
)
from rdkit import Chem
import torch

---
## 1. PDB Standardization

The PDB standardizer cleans protein structures:
- Removes water molecules and nucleic acids
- Handles post-translational modifications (PTMs)
- Normalizes protonation states (HID/HIE/HIP → HIS)
- Reorders atoms by standard definitions
- Renumbers residues sequentially

In [ ]:
# Basic standardization
input_pdb = '10gs_protein.pdb'
output_pdb = '10gs_standardized.pdb'

standardizer = PDBStandardizer(
    remove_hydrogens=True,
    ptm_handling='base_aa'  # Convert PTMs to base amino acids
)
standardizer.standardize(input_pdb, output_pdb)
print(f"Standardized PDB saved to: {output_pdb}")

### PTM Handling Modes

| Mode | Description | Use Case |
|------|-------------|----------|
| `'base_aa'` | Convert PTMs to base amino acids (SEP→SER) | ESM models, MD simulations |
| `'preserve'` | Keep PTM atoms intact | Protein-ligand modeling |
| `'remove'` | Remove PTM residues entirely | Standard AA-only analysis |

In [ ]:
# Using convenience function
standardize_pdb(input_pdb, '10gs_preserved.pdb', ptm_handling='preserve')
print("PTM-preserved version saved")

---
## 2. Protein Featurization

Extract structural features at atom and residue levels.

In [ ]:
# Initialize ProteinFeaturizer
protein_featurizer = ProteinFeaturizer(output_pdb)

print(f"Number of residues: {protein_featurizer.num_residues}")

### 2.1 Sequence Extraction

In [ ]:
# Get sequences by chain
sequences = protein_featurizer.get_sequence_by_chain()

for chain_id, sequence in sequences.items():
    print(f"Chain {chain_id}: {sequence[:50]}{'...' if len(sequence) > 50 else ''}")
    print(f"  Length: {len(sequence)} residues\n")

### 2.2 Atom-level Graph Features

In [ ]:
# Get atom-level graph (with SASA)
atom_node, atom_edge = protein_featurizer.get_atom_graph(distance_cutoff=4.0)

print("Atom-level Graph Features:")
print(f"  Node features shape: {atom_node['node_feats'].shape}")
print(f"  Edge features shape: {atom_edge['edge_feats'].shape}")
print(f"  Edge index shape: {atom_edge['edge_index'].shape}")
print(f"  Number of edges: {atom_edge['edge_index'].shape[1]}")

### 2.3 Residue-level Features

In [ ]:
# Get residue-level features
res_node, res_edge = protein_featurizer.get_features(distance_cutoff=8.0)

print("Residue-level Features:")
print(f"  Node features: {res_node.keys()}")
print(f"  Edge features: {res_edge.keys()}")

### 2.4 Contact Map

In [ ]:
# Get contact map with different thresholds
contacts = protein_featurizer.get_contact_map(cutoff=8.0)

print("Contact Map:")
print(f"  Adjacency matrix shape: {contacts['adjacency_matrix'].shape}")
print(f"  Number of contacts: {len(contacts['edges'][0])}")

---
## 3. Hierarchical Featurization with ESM Embeddings

Extract comprehensive features including ESM language model embeddings.

**Note:** This requires ESM models to be installed and may need GPU.

In [ ]:
# Initialize HierarchicalFeaturizer (loads ESM models)
# Skip if ESM not available
try:
    hier_featurizer = HierarchicalFeaturizer(
        esmc_model='esmc_600m',
        esm3_model='esm3-open',
        esm_device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    ESM_AVAILABLE = True
    print(f"HierarchicalFeaturizer initialized (device: {hier_featurizer.esm_device})")
except Exception as e:
    ESM_AVAILABLE = False
    print(f"ESM models not available: {e}")

In [ ]:
if ESM_AVAILABLE:
    # Extract all features
    data = hier_featurizer.featurize(output_pdb)
    
    print("Hierarchical Features:")
    print(f"\nAtom-level (integer indices for nn.Embedding):")
    print(f"  atom_tokens: {data.atom_tokens.shape} (0-186, 187 classes)")
    print(f"  atom_elements: {data.atom_elements.shape} (0-7, 8 classes)")
    print(f"  atom_residue_types: {data.atom_residue_types.shape} (0-21, 22 classes)")
    print(f"  atom_coords: {data.atom_coords.shape}")
    print(f"  atom_sasa: {data.atom_sasa.shape}")
    
    print(f"\nResidue-level:")
    print(f"  residue_features: {data.residue_features.shape} (76-dim scalar)")
    print(f"  residue_vector_features: {data.residue_vector_features.shape} (31x3 vector)")
    
    print(f"\nESM Embeddings:")
    print(f"  esmc_embeddings: {data.esmc_embeddings.shape}")
    print(f"  esmc_bos/eos: {data.esmc_bos.shape}")
    print(f"  esm3_embeddings: {data.esm3_embeddings.shape}")
    print(f"  esm3_bos/eos: {data.esm3_bos.shape}")
    
    print(f"\nAtom-Residue Mapping:")
    print(f"  atom_to_residue: {data.atom_to_residue.shape}")
    print(f"  num_atoms: {data.num_atoms}, num_residues: {data.num_residues}")

In [ ]:
# Select subset of residues (e.g., binding pocket)
if ESM_AVAILABLE:
    pocket_residues = [10, 11, 12, 45, 46, 47, 100, 101, 102]
    pocket_data = data.select_residues(pocket_residues)
    
    print(f"Pocket subset:")
    print(f"  Selected residues: {len(pocket_residues)}")
    print(f"  Pocket atoms: {pocket_data.num_atoms}")
    print(f"  Pocket residue_features: {pocket_data.residue_features.shape}")

---
## 4. Molecule Featurization

Extract features from small molecules (ligands):
- **Descriptors**: 40 normalized molecular properties
- **Fingerprints**: 9 types (Morgan, MACCS, RDKit, etc.)
- **Graph Features**: 157D atom features, 66D bond features

In [ ]:
# Initialize from SDF file
ligand_file = '10gs_ligand.sdf'
suppl = Chem.SDMolSupplier(ligand_file, removeHs=False)
mol = suppl[0]

mol_featurizer = LigandFeatureFeaturizer(mol, hydrogen=True)

print(f"Ligand info:")
print(f"  SMILES: {mol_featurizer.input_smiles}")
print(f"  Num atoms: {mol_featurizer.num_atoms}")
print(f"  Num bonds: {mol_featurizer.num_bonds}")
print(f"  Num rings: {mol_featurizer.num_rings}")
print(f"  Has 3D: {mol_featurizer.has_3d}")

### 4.1 Molecular Descriptors and Fingerprints

In [ ]:
# Get all molecular-level features
features = mol_featurizer.get_feature()

print("Molecular Features:")
print(f"\nDescriptors (normalized 0-1):")
print(f"  descriptor: {features['descriptor'].shape}")

print(f"\nFingerprints:")
print(f"  maccs: {features['maccs'].shape}")
print(f"  morgan: {features['morgan'].shape}")
print(f"  morgan_counts: {features['morgan_counts'].shape}")
print(f"  rdkit_fp: {features['rdkit_fp'].shape}")
print(f"  atom_pair: {features['atom_pair'].shape}")
print(f"  topological_torsion: {features['topological_torsion'].shape}")
print(f"  pattern: {features['pattern'].shape}")
print(f"  layered: {features['layered'].shape}")

### 4.2 Graph Features (for GNN)

In [ ]:
# Get graph representation
node, edge, adj = mol_featurizer.get_graph()

print("Graph Features:")
print(f"\nNode (Atom) Features:")
print(f"  node_feats: {node['node_feats'].shape} (157-dim)")

print(f"\nEdge (Bond) Features:")
print(f"  edge_feats: {edge['edge_feats'].shape} (66-dim)")
print(f"  edge_index: {edge['edge_index'].shape}")

print(f"\nAdjacency Matrix: {adj.shape}")

### 4.3 From SMILES with Custom SMARTS

In [ ]:
# Initialize from SMILES with custom patterns
custom_patterns = {
    'carboxyl': 'C(=O)O',
    'hydroxyl': '[OH]',
    'amine': '[NX3;H2,H1;!$(NC=O)]'
}

aspirin_featurizer = LigandFeatureFeaturizer(
    "CC(=O)Oc1ccccc1C(=O)O",  # Aspirin SMILES
    hydrogen=False,
    custom_smarts=custom_patterns
)

node, edge, adj = aspirin_featurizer.get_graph()
print(f"Aspirin with custom SMARTS:")
print(f"  node_feats: {node['node_feats'].shape} (157 + {len(custom_patterns)} custom)")

# Check custom features separately
custom_feats = aspirin_featurizer.get_custom_smarts_features()
print(f"  Custom feature names: {custom_feats['names']}")

---
## 5. Protein-Ligand Interaction Features

Detect and featurize interactions between protein and ligand:
- Hydrogen bonds
- Salt bridges
- Pi-stacking
- Hydrophobic contacts
- And more...

In [ ]:
# Load protein and ligand as RDKit molecules
protein_mol = Chem.MolFromPDBFile(output_pdb, removeHs=False)
ligand_mol = Chem.MolFromMolFile(ligand_file.replace('.sdf', '.sdf'), removeHs=False)

# For SDF files, use SDMolSupplier
if ligand_mol is None:
    suppl = Chem.SDMolSupplier(ligand_file, removeHs=False)
    ligand_mol = suppl[0]

print(f"Protein atoms: {protein_mol.GetNumAtoms()}")
print(f"Ligand atoms: {ligand_mol.GetNumAtoms()}")

In [ ]:
# Initialize PLInteractionFeaturizer
pli_featurizer = PLInteractionFeaturizer(
    protein_mol,
    ligand_mol,
    distance_cutoff=4.5
)

print(f"PLInteractionFeaturizer initialized")
print(f"  Protein heavy atoms: {pli_featurizer.n_protein_atoms}")
print(f"  Ligand heavy atoms: {pli_featurizer.n_ligand_atoms}")

### 5.1 Detect Interactions

In [ ]:
# Detect specific interaction types
hbonds = pli_featurizer.detect_hydrogen_bonds()
salt_bridges = pli_featurizer.detect_salt_bridges()
pi_stacking = pli_featurizer.detect_pi_stacking()
hydrophobic = pli_featurizer.detect_hydrophobic()

print("Detected Interactions:")
print(f"  Hydrogen bonds: {len(hbonds)}")
print(f"  Salt bridges: {len(salt_bridges)}")
print(f"  Pi-stacking: {len(pi_stacking)}")
print(f"  Hydrophobic: {len(hydrophobic)}")

In [ ]:
# Get interaction summary
summary = pli_featurizer.get_interaction_summary()
print(summary)

### 5.2 Interaction Graph for GNN

In [ ]:
# Get interaction edges and features
edges, edge_features = pli_featurizer.get_interaction_edges()

print("Interaction Graph:")
print(f"  edges: {edges.shape} (protein_idx, ligand_idx pairs)")
print(f"  edge_features: {edge_features.shape} (73-dim features)")

In [ ]:
# Get complete interaction graph data
graph = pli_featurizer.get_interaction_graph()

print("\nComplete Interaction Graph:")
for key, value in graph.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: {value.shape}")
    elif isinstance(value, dict):
        print(f"  {key}: {value}")
    else:
        print(f"  {key}: {type(value).__name__}")

### 5.3 Pharmacophore Features

In [ ]:
# Get pharmacophore features for atoms
protein_pharm, ligand_pharm = pli_featurizer.get_atom_pharmacophore_features()

print("Pharmacophore Features:")
print(f"  protein_pharm: {protein_pharm.shape} (7 pharmacophore types)")
print(f"  ligand_pharm: {ligand_pharm.shape} (7 pharmacophore types)")
print(f"\nPharmacophore types: HBD, HBA, Positive, Negative, Aromatic, Hydrophobic, Halogen")

---
## 6. Summary

This notebook demonstrated:

| Feature | Class | Key Methods |
|---------|-------|-------------|
| PDB Standardization | `PDBStandardizer` | `standardize()` |
| Protein Features | `ProteinFeaturizer` | `get_atom_graph()`, `get_features()`, `get_sequence_by_chain()` |
| Hierarchical + ESM | `HierarchicalFeaturizer` | `featurize()`, `select_residues()` |
| Ligand Features | `LigandFeatureFeaturizer` | `get_feature()`, `get_graph()` |
| Interactions | `PLInteractionFeaturizer` | `get_interaction_edges()`, `detect_*()` |

All features are returned as **PyTorch tensors** ready for use in deep learning models.

In [ ]:
# Cleanup
import os
for f in ['10gs_standardized.pdb', '10gs_preserved.pdb']:
    if os.path.exists(f):
        os.remove(f)
        print(f"Removed: {f}")